In [ ]:
# Instal dependency
!pip install open_clip_torch

In [1]:
# Import module
import os
import open_clip
import glob
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm

# Parse data path

In [2]:
keyframes_dir = 'E:/keyframe'
all_keyframe_paths = dict()

# Lấy danh sách các keyframes chỉ xử lý các thư mục có đuôi '_extra'
for part in sorted(os.listdir(keyframes_dir)):
    if not part.endswith('_extra1'):  # Chỉ xử lý thư mục có đuôi '_extra'
        continue
    data_part = part.split('_')[-2]  # Ví dụ: L01, L02 từ L01_extra, L02_extra
    all_keyframe_paths[data_part] = dict()

# Duyệt qua các keyframes đã lọc
for data_part in sorted(all_keyframe_paths.keys()):
    data_part_path = f'{keyframes_dir}/{data_part}_extra1'  # Đường dẫn thư mục _extra
    video_dirs = sorted(os.listdir(data_part_path))
    video_ids = [video_dir.split('_')[-1] for video_dir in video_dirs]
    for video_id, video_dir in zip(video_ids, video_dirs):
        keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.jpg'))
        all_keyframe_paths[data_part][video_id] = keyframe_paths

In [3]:
all_keyframe_paths

{'L07': {'V001': ['E:/keyframe/L07_extra1/V001\\000000.jpg',
   'E:/keyframe/L07_extra1/V001\\000002.jpg',
   'E:/keyframe/L07_extra1/V001\\000004.jpg',
   'E:/keyframe/L07_extra1/V001\\000006.jpg',
   'E:/keyframe/L07_extra1/V001\\000007.jpg',
   'E:/keyframe/L07_extra1/V001\\000018.jpg',
   'E:/keyframe/L07_extra1/V001\\000029.jpg',
   'E:/keyframe/L07_extra1/V001\\000041.jpg',
   'E:/keyframe/L07_extra1/V001\\000042.jpg',
   'E:/keyframe/L07_extra1/V001\\000125.jpg',
   'E:/keyframe/L07_extra1/V001\\000208.jpg',
   'E:/keyframe/L07_extra1/V001\\000292.jpg',
   'E:/keyframe/L07_extra1/V001\\000293.jpg',
   'E:/keyframe/L07_extra1/V001\\000313.jpg',
   'E:/keyframe/L07_extra1/V001\\000333.jpg',
   'E:/keyframe/L07_extra1/V001\\000353.jpg',
   'E:/keyframe/L07_extra1/V001\\000354.jpg',
   'E:/keyframe/L07_extra1/V001\\000377.jpg',
   'E:/keyframe/L07_extra1/V001\\000400.jpg',
   'E:/keyframe/L07_extra1/V001\\000423.jpg',
   'E:/keyframe/L07_extra1/V001\\000424.jpg',
   'E:/keyframe/L07

# Model

In [4]:
# ##### Load Model #####
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', device=device, pretrained='openai')

In [5]:
bs = 512
save_dir = 'E:/clip-32-feature'
if not os.path.exists(save_dir):
  os.mkdir(save_dir)

for key, video_keyframe_paths in tqdm(all_keyframe_paths.items()):
    if key not in ['L07']:
        continue
    video_ids = sorted(video_keyframe_paths.keys())

    
    if not os.path.exists(os.path.join(save_dir, key)):
        os.mkdir(os.path.join(save_dir, key))
    
    for video_id in tqdm(video_ids, desc=f"Processing {key}_{video_id}"):
        # if video_id < 'V011':
        #     continue
        video_feats = []
        video_keyframe_path = video_keyframe_paths[video_id]
        for i in range(0, len(video_keyframe_path), bs):
            # Support batchsize inferencing
            images = []
            image_paths = video_keyframe_path[i:i+bs]
            for image_path in image_paths:
                image = preprocess(Image.open(image_path)).unsqueeze(0)
                images.append(image)
            images = torch.cat(images).to(device)

            with torch.no_grad(), torch.cuda.amp.autocast():
                image_feats = model.encode_image(images)
            image_feats /= image_feats.norm(dim=-1, keepdim=True)

            for b in range(image_feats.shape[0]):
                video_feats.append(image_feats[b].detach().cpu().numpy().astype(np.float32).flatten())
        
        np.save(f'{save_dir}/{key}/{video_id}.npy', video_feats)

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Admin\AppData\Local\Temp\ipykernel_16644\1855018013.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
d:\anaconda\envs\py310\lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
100%|██████████| 1/1 [30:38<00:00, 1838.19s/it]
